In [1]:
# Cell 1: Imports and configuration
import pickle
import json
import requests
from typing import List, Dict
import random
from tqdm import tqdm
import re
from transformers import AutoTokenizer

# vLLM API configuration
VLLM_URL = "http://0.0.0.0:8000/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}

GENERATION_CONFIG = {
    "model": "meta-llama/llama-3.1-405b-instruct-fp8",
    #"model": "meta-llama/llama-3.3-70b-instruct",
    #"model": "meta-llama/llama-3.1-8b-instruct",
    "temperature": 0.8,
    "max_tokens": 2000,  # Increased for multi-turn conversations
    #"max_tokens": 4096,  # Increased for multi-turn conversations
    "top_p": 0.95
}

# Load processed conversations
#with open('data/processed_conversations.pkl', 'rb') as f:
#    conversations = pickle.load(f)

In [ ]:
from datasets import load_dataset

# Load private dataset (requires authentication)
dataset = load_dataset("mikeriess/LM_anonymized")
df = dataset['train'].to_pandas()
df

In [ ]:
conversations = dict(zip(df['conversation_id'], df['messages']))
conversations

In [ ]:
df.head()

# Testing on a single dialogue

In [5]:
custom_prompt = """Læs følgende kontekst, og opsummer problemstilling der tales om. 
    
    Kontekst:
    {context}
    
    Opsummering:"""


In [27]:

# Define prompt template
custom_prompt = """Du er en dansk samtalegenerator. Generer en realistisk dialog med flere udvekslinger mellem 2 personer på dansk, inspireret af følgende samtale som kontekst.
    
    VIGTIGT: Den første besked SKAL være et indledende spørgsmål som en bruger ønsker at få hjælp til. Svarene skal være i en venlig tone og aldrig indeholde URLs. Svarene må gerne indeholde relevante søgeord.
    
    Dialogen skal have mindst 1 udveksling frem og tilbage, men gerne flere hvis konteksten giver anledning til det.
    Formater hver besked som:
    Person 1: [spørgsmål]
    Person 2: [svar]
    Person 1: [opfølgende spørgsmål eller udsagn]
    Person 2: [svar]
    ...
    
    Kontekst:
    {context}
    
    Generer en ny, anderledes dialog omhandlende samme problemstilling:"""

In [ ]:
from tools.generator_utils import generate_dialogue_from_prompt

# Generation configuration
#GENERATION_CONFIG = {
#    "model": "meta-llama/llama-3.1-8b-instruct",
#    "temperature": 0.5,
#    "max_tokens": 2000,
#    "top_p": 0.95
#}


# Generate synthetic dialogue
conversation_id = 176
conversation = {'conversation_id': conversation_id, 'messages': conversations[conversation_id]}


# Format context
context = f"{conversation['messages'][0]['poster_id']}: {conversation['messages'][0]['text']}"

# Generate dialogue
generated_dialogue = generate_dialogue_from_prompt(
    prompt=custom_prompt.format(context=context),
    generation_config=GENERATION_CONFIG
)

if generated_dialogue:
    print("\nGenerated Dialogue:")
    print(generated_dialogue)

In [ ]:
generated_dialogue

# Multiple dialogues:

In [ ]:
from tools.generator_utils import generate_dataset

# Generation configuration
#GENERATION_CONFIG = {
#    "model": "meta-llama/llama-3.1-8b-instruct",
#    "temperature": 0.75,
#    "max_tokens": 2000,
#    "top_p": 0.95
#}

# To create a dataset of summaries:
PROMPT_TEMPLATE = """Læs følgende kontekst, og opsummer problemstilling der tales om. 
    
    Kontekst:
    {context}
    
    Opsummering:"""

# To create a dataset of dialogues:
PROMPT_TEMPLATE = """Du er en dansk samtalegenerator. Generer en realistisk dialog med flere udvekslinger mellem 2 personer på dansk, inspireret af følgende samtale som kontekst.
    
    VIGTIGT: Den første besked SKAL være et indledende spørgsmål som en bruger ønsker at få hjælp til. Stil spørgsmålet i første besked sådan, at det giver kontekst for problemet.

    Dialogen skal have mindst 1 udveksling frem og tilbage, men gerne flere hvis konteksten giver anledning til det.
    Formater hver besked som:
    Person 1: [spørgsmål]
    Person 2: [svar]
    Person 1: [opfølgende spørgsmål eller udsagn]
    Person 2: [svar]
    ...
    
    Kontekst:
    {context}
    
    Generer en ny, anderledes dialog omhandlende samme problemstilling:"""

# Generate dataset
generated_dataset = generate_dataset(
    conversations=conversations,
    generation_config=GENERATION_CONFIG,
    prompt_template=PROMPT_TEMPLATE,
    is_conversation=True, # whether or not to parse the generated dialogue into a list of messages
    max_conversations=10,  # Limit for testing
    min_messages=2
)


In [ ]:
# Print samples
conv_sample_id = next(iter(generated_dataset))
print("\nSample conversation:")
print(generated_dataset[conv_sample_id]["generated_output"])
print("\nParsed messages:")
for msg in generated_dataset[conv_sample_id]["parsed_messages"]:
    print(f"Post {msg['post_number']} (User {msg['poster_id']}): {msg['text']}")

#summary_sample_id = next(iter(summary_dataset))
#print("\nSample summary:")
#print(summary_dataset[summary_sample_id]["generated_output"])


# Print a sample
#sample_id = next(iter(generated_dataset))
#print(f"\nSample generated dialogue for conversation {sample_id}:")
#print(generated_dataset[sample_id]["generated_dialogue"])

### Inspect the dataset:

In [ ]:
generated_dataset["176"].keys()

In [ ]:
generated_dataset["176"]["original_messages"]

In [ ]:
generated_dataset["176"]["parsed_messages"]

In [ ]:
generated_dataset["176"]["generated_output"]

In [ ]:
generated_dataset["176"]["metadata"]

# Create dataset

In [ ]:
from tools.generator_utils import create_analysis_dataset, create_hf_dataset
import pandas as pd

# Create analysis dataset
model_name = "meta-llama/llama-3.1-8b-instruct"
analysis_dataset = create_analysis_dataset(generated_dataset, model_name)
analysis_dataset


In [ ]:

# Convert to HuggingFace Dataset
hf_dataset = create_hf_dataset(
    analysis_dataset=analysis_dataset,
    split_name="train",
    add_metadata=True
)

# Print dataset info
print("\nDataset structure:")
print(hf_dataset)


In [ ]:
df = hf_dataset["train"].to_pandas()
df.head()

In [ ]:
df.loc[1]["orig_messages"]

In [ ]:
df.loc[1]["synthetic_messages"]

## Store

In [ ]:
# Save dataset
hf_dataset.save_to_disk('data/synthetic_conversations')

# Optionally push to HuggingFace Hub
# hf_dataset.push_to_hub("mikeriess/LM_dialogues1", private=True, token=True)